In [2]:
import math
import time
import torch
import torch.nn as nn
import numpy as np
import scipy.sparse as sp
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden = 16                                     # hidden dimension
dropout = 0.5
lr = 0.01 
weight_decay = 5e-4
fastmode = 'store_true'

In [3]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

def normalize(mx):
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def  load_data(path="./", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))
    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), # read node labels
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)  # read node features
    labels = encode_onehot(idx_features_labels[:, -1])                        # one-hot encoding for labels
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)                
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),       # read edge information
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    features = normalize(features)                                            # feature normalization
    adj = normalize(adj + sp.eye(adj.shape[0]))                               # edge normalization

    idx_train = range(140)                                                    # training set
    idx_val = range(200, 500)                                                 # validation set
    idx_test = range(500, 1500)                                               # testing set

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)                               # get adjacency matrix

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)
                                                           
    return adj, features, labels, idx_train, idx_val, idx_test          

adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [37]:
class GraphConvolution(Module):                            
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        # Please implement this function
        output = torch.mm(input, self.weight)
        output = torch.mm(adj, output)
        return output + self.bias
    
    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class GCN(nn.Module):                                             # a two-layer GCN
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = torch.nn.functional.relu(self.gc1(x, adj))
        x = torch.nn.functional.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return torch.nn.functional.log_softmax(x, dim=1)          # softmax

model = GCN(nfeat=features.shape[1], nhid=hidden,
            nclass=labels.max().item() + 1,dropout=dropout)

In [38]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)                         
    loss_train = torch.nn.functional.nll_loss(output[idx_train], labels[idx_train])  # loss function
    acc_train = accuracy(output[idx_train], labels[idx_train])                       # training accuracy
    loss_train.backward()                                                            # backpropagation
    optimizer.step()                                                                 # update parameters

    if not fastmode:
        model.eval()
        output = model(features, adj)
        
    if epoch%50 == 0:
        loss_val = torch.nn.functional.nll_loss(output[idx_val], labels[idx_val])
        acc_val = accuracy(output[idx_val], labels[idx_val])
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
              'time: {:.4f}s'.format(time.time() - t))

In [39]:
def test():
    model.eval()
    output = model(features, adj)                     # features:(2708, 1433)   adj:(2708, 2708)
    loss_test = torch.nn.functional.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    
optimizer = torch.optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)

epochs = 500
for epoch in range(epochs):
    train(epoch)
test()

Epoch: 0001 loss_train: 1.9249 acc_train: 0.2929 loss_val: 1.8946 acc_val: 0.3500 time: 0.0169s
Epoch: 0051 loss_train: 1.3579 acc_train: 0.5714 loss_val: 1.4994 acc_val: 0.4967 time: 0.0050s
Epoch: 0101 loss_train: 0.7170 acc_train: 0.8500 loss_val: 1.0255 acc_val: 0.7433 time: 0.0050s
Epoch: 0151 loss_train: 0.4706 acc_train: 0.9357 loss_val: 0.8474 acc_val: 0.7533 time: 0.0060s
Epoch: 0201 loss_train: 0.3713 acc_train: 0.9500 loss_val: 0.7361 acc_val: 0.7767 time: 0.0100s
Epoch: 0251 loss_train: 0.3621 acc_train: 0.9643 loss_val: 0.7874 acc_val: 0.7900 time: 0.0060s
Epoch: 0301 loss_train: 0.2763 acc_train: 0.9786 loss_val: 0.7895 acc_val: 0.7700 time: 0.0060s
Epoch: 0351 loss_train: 0.2765 acc_train: 0.9571 loss_val: 0.7135 acc_val: 0.7967 time: 0.0080s
Epoch: 0401 loss_train: 0.3006 acc_train: 0.9357 loss_val: 0.7293 acc_val: 0.7833 time: 0.0060s
Epoch: 0451 loss_train: 0.2695 acc_train: 0.9500 loss_val: 0.7460 acc_val: 0.7667 time: 0.0050s
Test set results: loss= 0.6156 accuracy=